In [1]:
import pandas as pd
import numpy as np

#pd.set_option('display.max_rows', None)
#coaching_time_compare

In [2]:
#fitbit data
wear_time = pd.read_csv("D:\\EIC-Code\\00-Python\\2024-08-Objective-identification-on-mental-health_CNN_LSTM\\data\\fitbitWearTimeViaHR_merged.csv")
dailyactivity = pd.read_csv("D:\\EIC-Code\\00-Python\\2024-08-Objective-identification-on-mental-health_CNN_LSTM\\data\\dailyActivity_merged.csv")
dailyFitbitActiveZoneMinutes = pd.read_csv("D:\\EIC-Code\\00-Python\\2024-08-Objective-identification-on-mental-health_CNN_LSTM\\data\\dailyFitbitActiveZoneMinutes_merged.csv")
dailyHRV = pd.read_csv("D:\\EIC-Code\\00-Python\\2024-08-Objective-identification-on-mental-health_CNN_LSTM\\data\\fitbitDailyHRV_merged.csv")
fitbitBreathingRate = pd.read_csv("D:\\EIC-Code\\00-Python\\2024-08-Objective-identification-on-mental-health_CNN_LSTM\\data\\fitbitBreathingRate_merged.csv")
fitbitSkinTemperature = pd.read_csv("D:\\EIC-Code\\00-Python\\2024-08-Objective-identification-on-mental-health_CNN_LSTM\\data\\fitbitSkinTemperature_merged.csv")
sleepDay = pd.read_csv("D:\\EIC-Code\\00-Python\\2024-08-Objective-identification-on-mental-health_CNN_LSTM\\data\\sleepDay_merged.csv")
heartrate_15min = pd.read_csv("D:\\EIC-Code\\00-Python\\2024-08-Objective-identification-on-mental-health_CNN_LSTM\\data\\heartrate_15min_merged.csv")
sleepStageLogInfo = pd.read_csv("D:\\EIC-Code\\00-Python\\2024-08-Objective-identification-on-mental-health_CNN_LSTM\\data\\sleepStagesDay_merged.csv")

#survey data
survey_data = pd.read_csv("D:\\EIC-Code\\00-Python\\2024-08-Objective-identification-on-mental-health_CNN_LSTM\\data\\ABriefInterventionTo_DATA_2024-09-22_1540.csv")

In [3]:
dailyactivity.rename(columns = {'ActivityDate':'date'}, inplace=True)
wear_time.rename(columns = {'Day':'date'}, inplace=True)
dailyFitbitActiveZoneMinutes.rename(columns = {'Date':'date'}, inplace=True)
dailyHRV.rename(columns = {'SleepDay':'date'}, inplace=True)
fitbitBreathingRate.rename(columns = {'SleepDay':'date'}, inplace = True)
fitbitSkinTemperature.rename(columns = {'SleepDay':'date'}, inplace=True)
sleepDay.rename(columns = {'SleepDay':'date'}, inplace=True)
sleepStageLogInfo.rename(columns = {'SleepDay':'date'}, inplace=True)
heartrate_15min.rename(columns = {'Time':'date'}, inplace=True)

#transfer date
total_data = [wear_time, dailyactivity, dailyFitbitActiveZoneMinutes, dailyHRV, fitbitBreathingRate, 
              fitbitSkinTemperature, sleepDay, sleepStageLogInfo, heartrate_15min]

for d in total_data:
    d['date'] = pd.to_datetime(d['date'], errors = 'coerce')
    d['date'] = d['date'].dt.date #.strftime('%m/%d/%Y').str.replace('/0', '/')

#transfer heart_rate_15 min
heartrate_15min = heartrate_15min.groupby(by = ['Id','date']).mean().reset_index()

C:\Users\karri\AppData\Local\Temp\ipykernel_40192\987032928.py:16: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  d['date'] = pd.to_datetime(d['date'], errors = 'coerce')
C:\Users\karri\AppData\Local\Temp\ipykernel_40192\987032928.py:16: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  d['date'] = pd.to_datetime(d['date'], errors = 'coerce')
C:\Users\karri\AppData\Local\Temp\ipykernel_40192\987032928.py:16: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  d['date'] = pd.to_datetime(d['date'], errors = 'coerce')


In [4]:
merged1 = pd.merge(wear_time, dailyactivity, on=['Id','date'], how='left')  # inner join only keeps common dates
merged2 = pd.merge(merged1, sleepDay, on=['Id','date'], how='left')
merged3 = pd.merge(merged2, sleepStageLogInfo, on=['Id','date'], how='left')
merged4 = pd.merge(merged3, heartrate_15min, on=['Id','date'], how='left')
merged5 = pd.merge(merged4, dailyFitbitActiveZoneMinutes, on=['Id','date'], how='left')
merged6 = pd.merge(merged5, dailyHRV, on=['Id','date'], how='left')
merged7 = pd.merge(merged6, fitbitBreathingRate, on=['Id','date'], how='left')
merged_final = pd.merge(merged7, fitbitSkinTemperature, on=['Id','date'], how='left')

In [5]:

merged_final_10 = merged_final[(merged_final['TotalMinutesWearTime'] >= 10*60)] #10hour, what's the meaning of four days a week, is this for calendar week or four consecutive days
merged_0_3000steps = merged_final[(merged_final['TotalMinutesWearTime'] == 0) & (merged_final['TotalSteps'] >= 3000)]

merged_final = pd.concat([merged_final_10, merged_0_3000steps])
merged_final = merged_final.sort_values(by=['Id', 'date']) 

#rename ID
merged_final['Id'] = [items[-3:] for items in merged_final['Id']]
merged_final['Id'] = [int(digit) for digit in merged_final['Id']]

Survey data, find sepecific coaching time

In [7]:
#survey data preprocessing
survey_data['health_coach_survey_complete'].unique()

#select people who complete the second follow-up
survey_data_clean = survey_data[survey_data['weeks_followup_survey_complete'] == 2] #96 participants

In [8]:
#select variables from 
uses_vars_of_survey = ['record_id','current_status',
'demographics_age','demographics_sex','demographics_sexorient','demographics_ethnicity','demographics_immigration',
'demographics_race___1', 'demographics_race___2', 'demographics_race___3', 'demographics_race___4', 'demographics_race___5', 
'demographics_race___6', 'demographics_race___7', 'demographics_education', 'demographics_sorority',
'nervous_v1', 'down_v1', 'calm_v1', 'blue_v1', 'happy_v1',
'nervous_v2', 'down_v2', 'calm_v2', 'blue_v2', 'happy_v2',
'weeks_followup_survey_timestamp', 'weeks_followup_survey_96ac_timestamp'
]
select_survey = survey_data_clean[uses_vars_of_survey]

#rename the name of record_id
select_survey.rename(columns = {"record_id":'Id'}, inplace=True)

#reframe the date time format
select_survey['weeks_followup_survey_timestamp'] = pd.to_datetime(select_survey['weeks_followup_survey_timestamp'], errors = 'coerce' )
select_survey['weeks_followup_survey_timestamp'] = select_survey['weeks_followup_survey_timestamp'].dt.date
select_survey['weeks_followup_survey_96ac_timestamp'] = pd.to_datetime(select_survey['weeks_followup_survey_96ac_timestamp'], errors = 'coerce' )
select_survey['weeks_followup_survey_96ac_timestamp'] = select_survey['weeks_followup_survey_96ac_timestamp'].dt.date

C:\Users\karri\AppData\Local\Temp\ipykernel_40192\3305042889.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  select_survey.rename(columns = {"record_id":'Id'}, inplace=True)
C:\Users\karri\AppData\Local\Temp\ipykernel_40192\3305042889.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  select_survey['weeks_followup_survey_timestamp'] = pd.to_datetime(select_survey['weeks_followup_survey_timestamp'], errors = 'coerce' )
C:\Users\karri\AppData\Local\Temp\ipykernel_40192\3305042889.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slic

Merge Fitbit data with coaching time

In [9]:
fitbit_data = merged_final.copy()

#replace code 98 as 107 because the code of this participant is 107 in survey
fitbit_data.loc[fitbit_data['Id'] == 98, 'Id'] = 107

used_var = ['Id','date','TotalMinutesWearTime', 'TotalSteps', 'VeryActiveMinutes', 'FairlyActiveMinutes',
       'LightlyActiveMinutes', 'SedentaryMinutes', 'Calories', 'RestingHeartRate','TotalActiveZoneMinutes', 'FatBurnActiveZoneMinutes',
       'CardioActiveZoneMinutes', 'PeakActiveZoneMinutes', 'DailyRMSSD', 'DeepRMSSD',
       'AvgBreathsPerMinute', 'LightSleepAvgBreathsPerMinute','DeepSleepAvgBreathsPerMinute', 'REMSleepAvgBreathsPerMinute',
       'TotalMinutesAsleep_x', 'TotalTimeInBed_x', 'TotalTimeAwake', 'TotalMinutesLight', 
       'TotalMinutesDeep','TotalMinutesREM', 'NightlyRelative', 'Value'] #delete 'Above', 'Below', Custom Zone; NightlyRelative: ski temparature

used_fitbit_data = fitbit_data[used_var]

used_fitbit_data.rename(columns = {'TotalMinutesAsleep_x':'TotalMinutesAsleep','TotalTimeInBed_x':'TotalMinutesinbed',
                                   'TotalTimeAwake': 'TotalMinutesAwake',
                                   'Value': 'HR',
                                   'NightlyRelative': 'skin_temparature'}, 
                                   inplace = True)

#select participants matched with survey
all_survey_id = select_survey['Id'].unique()
used_fitbit_data = used_fitbit_data[used_fitbit_data['Id'].isin(all_survey_id)]

#add another column Fitbit data
used_fitbit_data.insert(loc = 2, column = 'survey_date', value = np.nan)

C:\Users\karri\AppData\Local\Temp\ipykernel_40192\2496330559.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  used_fitbit_data.rename(columns = {'TotalMinutesAsleep_x':'TotalMinutesAsleep',


In [ ]:
#check the individual who did not complete second follow-up survey
select_survey[pd.isna(select_survey['weeks_followup_survey_96ac_timestamp'])]['Id']
# 40      42
# 46      48
# 67      69
# 107    109dd
# 112    114
# 123    125
# 129    131
# 132    134
# 149    151

In [10]:
def assign_survey_date(row, followup1_time, followup2_time):
    time = row['date']
    
    if pd.isna(followup2_time):
        if pd.notna(time) and time <= followup1_time:
            return "follow_up1"
    
    elif pd.notna(followup2_time):
        if pd.notna(time) and time <= followup2_time:
            return "follow_up2"
    
    return None

In [11]:
all_fitbit_id = used_fitbit_data['Id'].unique()

for i in range(len(all_fitbit_id)):

    fitbit_id = all_fitbit_id[i]

    #outcome variable
    individual_survey = select_survey[select_survey['Id']==fitbit_id]
    followup1_time = individual_survey['weeks_followup_survey_timestamp'].iloc[0] if not individual_survey['weeks_followup_survey_timestamp'].empty else pd.NaT
    followup2_time = individual_survey['weeks_followup_survey_96ac_timestamp'].iloc[0] if not individual_survey['weeks_followup_survey_96ac_timestamp'].empty else pd.NaT
    
    #predictors
    individual_fitbit = used_fitbit_data[used_fitbit_data['Id']==fitbit_id].copy()

    individual_fitbit['survey_date'] = individual_fitbit.apply(
        lambda row: assign_survey_date(row, followup1_time, followup2_time), axis=1
        )
    used_fitbit_data.loc[used_fitbit_data['Id'] == fitbit_id, 'survey_date'] = individual_fitbit['survey_date'] #replace with new define


used_fitbit_data = used_fitbit_data.dropna(subset=['survey_date']) #4957

In [12]:
#time sequence complement
time_sequency = used_fitbit_data.copy()
all_fitbit_id = time_sequency['Id'].unique()
new_data_frame = pd.DataFrame()

for i in range(len(all_fitbit_id)):

    fitbit_id = all_fitbit_id[i]

    sorted_individual = time_sequency[time_sequency['Id'] == fitbit_id].sort_values(by = 'date')
    
    #outcome variable
    individual_survey = select_survey[select_survey['Id']==fitbit_id]
    followup1_time = individual_survey['weeks_followup_survey_timestamp'].iloc[0] if not individual_survey['weeks_followup_survey_timestamp'].empty else pd.NaT
    followup2_time = individual_survey['weeks_followup_survey_96ac_timestamp'].iloc[0] if not individual_survey['weeks_followup_survey_96ac_timestamp'].empty else pd.NaT
    
    start_date = sorted_individual['date'].iloc[0]

    if 'follow_up1' in sorted_individual['survey_date'].values:
        end_date = followup1_time
    else:
        end_date = followup2_time
    
    full_date_range = pd.date_range(start=start_date, end=end_date)

    sorted_individual.set_index('date', inplace=True)
    
    sorted_individual_reindexed = sorted_individual.reindex(full_date_range).reset_index() #reset the wrong index
    sorted_individual_reindexed.rename(columns = {'index':'date'}, inplace = True)

    if 'follow_up1' in sorted_individual_reindexed['survey_date'].values:
        sorted_individual_reindexed['survey_date'] = 'follow_up1'
        sorted_individual_reindexed['Id'] = fitbit_id
    else:
        sorted_individual_reindexed['survey_date'] = 'follow_up2'
        sorted_individual_reindexed['Id'] = fitbit_id

    new_data_frame = pd.concat([new_data_frame, sorted_individual_reindexed])

In [34]:
final_data_frame = pd.DataFrame()

for i1 in range(len(all_fitbit_id)):

    fitbit_id = all_fitbit_id[i1]
    select_data = new_data_frame[new_data_frame['Id'] == fitbit_id]

    len_select_data = len(select_data)

    process_series = pd.Series(dtype = 'object')

    for i2 in range(len_select_data):

        row_select_data = select_data.iloc[i2].copy()
        row_select_data.index = [str(col) + '_' + str(i2) for col in row_select_data.index] #revise the name
        process_series = pd.concat([process_series, row_select_data])

    trans_process_series = pd.DataFrame(process_series).T

    final_data_frame = pd.concat([final_data_frame, trans_process_series], axis = 0, ignore_index=True)
    

In [35]:
final_data_frame = final_data_frame.set_index(['Id_0','date_0'])#set ID and date as index
final_data_frame = final_data_frame.drop(final_data_frame.filter(regex = 'date|Id|survey_date|TotalMinutesWearTime').columns, axis = 1) #delete column with the same format
final_data_frame.reset_index(inplace=True) #reset_index
final_data_frame.rename(columns = {'Id_0':'Id', 'date_0':'startdate'}, inplace = True) #renamei d

In [15]:
#final_data_frame.to_csv('fitbit_final_output_time_series.csv')

Merge fitbit data with survey data

In [47]:
#merge survey data with fitbit data
fitbit_survey = pd.merge(final_data_frame, select_survey, on = 'Id', how = 'left')

In [50]:
fitbit_survey['enddate'] = np.nan

all_ids = fitbit_survey['Id'].unique()

for i in range(len(all_ids)):

    id = all_ids[i]
    individual_data = fitbit_survey[fitbit_survey['Id'] == id]

    if pd.isna(individual_data['weeks_followup_survey_96ac_timestamp'].iloc[0]):
        fitbit_survey.loc[fitbit_survey['Id'] == id, 'enddate'] = individual_data['weeks_followup_survey_timestamp'].iloc[0]
        print(True)
    else:
        fitbit_survey.loc[fitbit_survey['Id'] == id, 'enddate']= individual_data['weeks_followup_survey_96ac_timestamp'].iloc[0]

fitbit_survey.drop(columns = ['weeks_followup_survey_timestamp', 'weeks_followup_survey_96ac_timestamp'], inplace = True) #drop another two columns

,Id,date,TotalSteps_0,VeryActiveMinutes_0,FairlyActiveMinutes_0,LightlyActiveMinutes_0,SedentaryMinutes_0,Calories_0,RestingHeartRate_0,TotalActiveZoneMinutes_0,...,calm_v1,blue_v1,happy_v1,nervous_v2,down_v2,calm_v2,blue_v2,happy_v2,weeks_followup_survey_timestamp,weeks_followup_survey_96ac_timestamp
0,5,2023-11-15,18236.0,71.0,25.0,225.0,539.0,2747.0,66.0,175.0,...,2.0,5.0,2.0,5.0,5.0,1.0,5.0,2.0,2023-12-06,2024-01-07
1,7,2023-10-18,16938.0,94.0,61.0,219.0,505.0,3346.0,54.0,193.0,...,5.0,6.0,2.0,3.0,4.0,3.0,4.0,2.0,2023-12-16,2024-01-17
2,8,2023-10-07,21238.0,44.0,45.0,428.0,505.0,3577.0,64.0,102.0,...,4.0,6.0,2.0,5.0,6.0,2.0,6.0,2.0,2023-11-03,2023-12-04
3,13,2023-10-08,9058.0,8.0,11.0,343.0,186.0,2843.0,81.0,56.0,...,3.0,4.0,4.0,1.0,1.0,4.0,6.0,3.0,2023-11-20,2024-04-10
4,14,2023-10-09,4038.0,16.0,39.0,346.0,474.0,3148.0,67.0,24.0,...,5.0,1.0,5.0,2.0,1.0,5.0,1.0,5.0,2023-11-19,2023-12-26
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
90,156,2024-04-10,9369.0,5.0,8.0,348.0,1079.0,1513.0,NaN,NaN,...,1.0,4.0,1.0,1.0,3.0,5.0,2.0,3.0,2024-05-07,2024-06-07
91,162,2024-04-25,2880.0,0.0,0.0,106.0,676.0,2138.0,64.0,7.0,...,3.0,5.0,3.0,2.0,2.0,5.0,4.0,5.0,2024-06-10,2024-07-11
92,164,2024-05-01,4652.0,64.0,11.0,171.0,822.0,2963.0,66.0,116.0,...,2.0,6.0,3.0,6.0,6.0,3.0,6.0,2.0,2024-05-27,2024-06-24
93,165,2024-05-16,5379.0,6.0,3.0,147.0,937.0,1768.0,NaN,NaN,...,2.0,5.0,2.0,4.0,4.0,4.0,4.0,4.0,2024-07-11,2024-08-08


In [81]:
#how to remvoe column 1 to -2

,Id,date,TotalSteps_0,VeryActiveMinutes_0,FairlyActiveMinutes_0,LightlyActiveMinutes_0,SedentaryMinutes_0,Calories_0,RestingHeartRate_0,TotalActiveZoneMinutes_0,...,down_v1,calm_v1,blue_v1,happy_v1,nervous_v2,down_v2,calm_v2,blue_v2,happy_v2,enddate
0,5,2023-11-15,18236.0,71.0,25.0,225.0,539.0,2747.0,66.0,175.0,...,6.0,2.0,5.0,2.0,5.0,5.0,1.0,5.0,2.0,2024-01-07
1,7,2023-10-18,16938.0,94.0,61.0,219.0,505.0,3346.0,54.0,193.0,...,6.0,5.0,6.0,2.0,3.0,4.0,3.0,4.0,2.0,2024-01-17
2,8,2023-10-07,21238.0,44.0,45.0,428.0,505.0,3577.0,64.0,102.0,...,6.0,4.0,6.0,2.0,5.0,6.0,2.0,6.0,2.0,2023-12-04
3,13,2023-10-08,9058.0,8.0,11.0,343.0,186.0,2843.0,81.0,56.0,...,4.0,3.0,4.0,4.0,1.0,1.0,4.0,6.0,3.0,2024-04-10
4,14,2023-10-09,4038.0,16.0,39.0,346.0,474.0,3148.0,67.0,24.0,...,1.0,5.0,1.0,5.0,2.0,1.0,5.0,1.0,5.0,2023-12-26
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
90,156,2024-04-10,9369.0,5.0,8.0,348.0,1079.0,1513.0,NaN,NaN,...,4.0,1.0,4.0,1.0,1.0,3.0,5.0,2.0,3.0,2024-06-07
91,162,2024-04-25,2880.0,0.0,0.0,106.0,676.0,2138.0,64.0,7.0,...,6.0,3.0,5.0,3.0,2.0,2.0,5.0,4.0,5.0,2024-07-11
92,164,2024-05-01,4652.0,64.0,11.0,171.0,822.0,2963.0,66.0,116.0,...,6.0,2.0,6.0,3.0,6.0,6.0,3.0,6.0,2.0,2024-06-24
93,165,2024-05-16,5379.0,6.0,3.0,147.0,937.0,1768.0,NaN,NaN,...,5.0,2.0,5.0,2.0,4.0,4.0,4.0,4.0,4.0,2024-08-08


In [67]:
#survey_fitbit.to_csv('final_survey_fitbit_time_series.csv')
fitbit_survey.iloc[0]['enddate']

nan